## Лекция 4 - Элементы функционального программирования в Python

- синтаксис функций
- LEGB
- итераторы и генераторы
- лямбда-функции
- filter, map, reduce, zip
- модуль itertools (is back)
- декораторы
- модуль functools

In [1]:
# пример простейших функций
# (уже не ново, т.к. видели примеры в пакете algo)

def foo(x):
    return x + 10
   
magic_number = foo(63)
magic_number

73

In [2]:
def bar():
    print('Hello')
    
bar()    

Hello


In [3]:
# функция может возвращать все что угодно:
# удобно, т.к. можно вернуть несколько значений сразу
# (по сути кортеж)
def arithmetics(a, b):
    ''' функция возвращает сумму, разность и произведение чисел '''
    summa = a + b
    diff = a - b
    mult = a * b
    return summa, diff, mult

s = arithmetics(10, 15)
s

(25, -5, 150)

In [4]:
_, difference, _ = arithmetics(7, 9)
difference

-2

In [5]:
# Синтаксис *args, **kwargs:
def pretty_print(text, *args, **kwargs):
    # отобразить "шапку"
    for cnt in args:
        print(kwargs['fill'] * cnt)
        
    # вывести собственно текст
    print(text)
    
    # отобразить футер, если параметр footer=True
    if kwargs['footer']:
        for cnt in reversed(args):
            print(kwargs['fill'] * cnt)
    
    # args kwargs
    print()        
    print('args:')
    print(args)
    print('kwargs:')
    print(kwargs)

    
pretty_print('Hello', 20, 10, 5, fill='~', footer=True)

~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~
~~~~~
Hello
~~~~~
~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~

args:
(20, 10, 5)
kwargs:
{'fill': '~', 'footer': True}


In [6]:
param_list = [50, 25, 10]

# param_list сам собой не развернется в последовательность параметров
# это будет один параметр - список:
pretty_print('Hello!', param_list, fill='~', footer=False)

TypeError: can't multiply sequence by non-int of type 'list'

In [7]:
# чтобы список трактовался как список параметров, пишем так:
pretty_print('Hello!', *param_list, fill='~', footer=False)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~
Hello!

args:
(50, 25, 10)
kwargs:
{'fill': '~', 'footer': False}


In [8]:
def print_chars(char, count):
    print(char * count)
    
print_chars('X', 10)

# параметры можно указать еще таким образом:
print_chars(count=5, char='*')

XXXXXXXXXX
*****


In [9]:
# демонстрация нюанса (список-параметр создается только единожды)
def append_item(item, arr=[]):
    arr.append(item)
    return arr

# здксь без эксцессов:
print(append_item(42, [1,2,3]))
print(append_item(73, [1,2,3]))

[1, 2, 3, 42]
[1, 2, 3, 73]


In [10]:
# а здесь - с эксцессами:
print(append_item(42))
print(append_item(73))

[42]
[42, 73]


In [11]:
# корректный обход нюанса
def append_item(item, arr=None):
    if arr is None:
        arr = []
    arr.append(item)
 
    return arr

print(append_item(42))
print(append_item(73))

[42]
[73]


### Функция - это first class объект

In [12]:
# очень важно: функция - это first class объект!
dir(pretty_print)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [13]:
# немного поинстроспектируем:
print('Квалифицированное имя функции:', pretty_print.__qualname__)
print('Переменные в функции:', pretty_print.__code__.co_varnames)

Квалифицированное имя функции: pretty_print
Переменные в функции: ('text', 'args', 'kwargs', 'cnt')


In [14]:
# эквивалентно foo(10)
foo.__call__(10)

20

In [15]:
# таким образом, 1) функции можно присваивать переменным:
func = foo
func(100)

110

In [16]:
# 2) можно делать массив из функций
#    и последовательно выполнять их:
import math

funcs = [foo, math.factorial]

for func in funcs:
    print(func.__qualname__, func(5))

foo 15
factorial 120


In [17]:
# 3) можно передавать функцию в функцию:
def process_elementwise(seq, func):
    return [func(s) for s in seq]

words = ['Fundamentals', 'of', 'brainwashing...']
process_elementwise(words, str.__len__)

[12, 2, 15]

In [18]:
process_elementwise(words, str.upper)

['FUNDAMENTALS', 'OF', 'BRAINWASHING...']

In [19]:
nums = [1, -3, 5, -7, -9]
process_elementwise(nums, abs)

[1, 3, 5, 7, 9]

In [20]:
process_elementwise(nums, foo)

[11, 7, 15, 3, 1]

### LEGB (области видимости)

- <b>L</b>, Local — все локальные имена: имена, любым способом присвоенные внутри функции (объявленных как ```def``` или ```lambda```), и не помеченные ключевым словом ```global``` в этой функции.


- <b>E</b>, Enclosing function locals — все имена в локальной области видимости всех замыкающих функций (объявленных как ```def``` или ```lambda```), от самой вложенной ко внешней.


- <b>G</b>, Global (на уровне модуля) — имена, глобальные для модуля или помеченные ключевым словом ```global``` внутри конструкции ```def``` где-либо в файле.


- <b>B</b>, Built-in (Python) — Предопределенные глобальные имена Python, которые списком содержатся в переменной ```__builtins__``` (которые, в свою очердь, берутся из модуля ```builtins```): ```print, open, reversed, ValueError``` и т.д.

In [21]:
x = 73

def wrong_scope():
    print(x)    # это сработает
    x += 1      # здесь бросит исключение
    
wrong_scope()    

UnboundLocalError: local variable 'x' referenced before assignment

In [22]:
x = 73

def scope_ok():
    global x
    print(x)
    x += 1
    
scope_ok()

print(x)

73
74


In [23]:
# более полная демонстрация LEGB, функции locals(),
# а также замыканий

x = 42
y = 73

# замыкающая функция
def enclosing_func():
    x = 500
    y = 1000
    print('Enclosing function locals:', locals())
    
    # внутренняя (вложенная) функция
    def inner_func():
        global x
        print('Inner func x, y:', x, y)
        print('Inner func locals:', locals())
        x += 1

    return inner_func

func = enclosing_func()
func()

print('Global x, y:', x, y)

Enclosing function locals: {'y': 1000, 'x': 500}
Inner func x, y: 42 1000
Inner func locals: {'y': 1000}
Global x, y: 43 73


In [24]:
# демонстрация использования ключевого слова nonlocal:

def enclosing_func():
    value = 0
    # вложенная функция наращивает value замыкающей функции
    def inner_func():
        nonlocal value
        value += 1
        return value
    return inner_func

id_generator = enclosing_func()

print(id_generator())
print(id_generator())

1
2


In [25]:
# Версия Python 2 (где нет nonlocal)

# для эмуляции nonlocal можно завести переменную mutable типа, 
# например, список из одного элемента:

def enclosing_func():
    value = [0]
    def inner_func():
        value[0] += 1
        return value[0]
    return inner_func

id_generator = enclosing_func()

print(id_generator())
print(id_generator())

1
2


In [26]:
n = 1

# переменные for-циклов "протекают" в глобальное пространство имен
for n in range(5):
    print(n, '(в цикле)')
        
print(n, '(в глобальной области видимости)')

0 (в цикле)
1 (в цикле)
2 (в цикле)
3 (в цикле)
4 (в цикле)
4 (в глобальной области видимости)


In [27]:
# В данном случае "протекания" не будет:

i = 1
print([i for i in range(5)])
print(i, '(в глобальной области видимости)')

[0, 1, 2, 3, 4]
1 (в глобальной области видимости)


### Итераторы и генераторы:
- итератор - это концепция (любой объект, имеющий методы ```next()``` и ```__iter__()```)
- генератор - языковое средство (объект вокруг функции с ```yield```)

Любой генератор является итератором, но не наоборот.

In [28]:
def odd_iterator(x):
    for i in range(1, x, 2):
        yield i

for x in odd_iterator(10):
    print(x)

1
3
5
7
9


In [29]:
it = odd_iterator(10)
it

<generator object odd_iterator at 0x02D7A450>

In [30]:
print(next(it))
print(next(it))

1
3


In [31]:
try:
    it.throw(ValueError('Iterator problems!'))
except ValueError as err:
    print('Caught:', err)

it.close()

Caught: Iterator problems!


In [32]:
next(it)

StopIteration: 

In [33]:
dir(it)

['__class__',
 '__del__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__iter__',
 '__le__',
 '__lt__',
 '__name__',
 '__ne__',
 '__new__',
 '__next__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'close',
 'gi_code',
 'gi_frame',
 'gi_running',
 'gi_yieldfrom',
 'send',
 'throw']

In [34]:
# Но это еще не все:
# есть также метод send, который позволяет делать т.н. корутины
def coroutine_example():
    while True:
        x = yield
        print('Got new number:', x)
        
cor = coroutine_example()
next(cor)

cor.send(2)
cor.send(5)
cor.send(10)

cor.close()

Got new number: 2
Got new number: 5
Got new number: 10


In [35]:
# предыдущий пример был игрушечный, но в асинхронных фреймворках
# корутины являются важнейшим элементом, и в них можно писать
# концептуально что-то вроде этого (чтобы это работало, нужен
# постоянно крутящийся цикл, в котором данные асинхронно 
# получаются и передаются, подробнее - в лекции №11):

# здесь логика генерации данных для обработки
def number_provider():
    for i in range(1, 11):
        yield i

# здесь логика получения данных и их обработки
def number_processor():
    while True:
        x = yield from number_provider
        print('Got new number:', x)

In [36]:
# в Python 3.3 появилась конструкция yield from:

def odd_number_provider(n):
    for i in range(1, n, 2):
        yield i

def even_number_provider(n):
    for i in range(2, n, 2):
        yield i
        
def number_generator(n):
    yield from odd_number_provider(n)
    yield from even_number_provider(n)
    
for number in number_generator(10):
    print(number)

1
3
5
7
9
2
4
6
8


In [37]:
# еще пример генератора (генерирующего цифры числа)
def generate_digits(x):
    while x > 0:
        yield x % 10
        x = x // 10

for digit in generate_digits(5716):
    print(digit)

6
1
7
5


In [38]:
l = [x for x in generate_digits(54321)]
l

[1, 2, 3, 4, 5]

In [39]:
i = generate_digits(4523)
print([next(i), next(i), next(i)])

[3, 2, 5]


In [40]:
# еще пример генератора (генерирующего перестановки n чисел)
import itertools

def permute(n):
    for perm in itertools.permutations(range(1, n+1)):
        yield perm
        
for p in permute(3):
    print(p)

(1, 2, 3)
(1, 3, 2)
(2, 1, 3)
(2, 3, 1)
(3, 1, 2)
(3, 2, 1)


In [41]:
# еще раз о разнице между range в python2 и python3
nums = range(1, 5)
nums

# в Python2 будет выведено [1, 2, 3, 4]
# аналог: nums = xrange(1, 5)

range(1, 5)

In [42]:
# range - специфический итерируемый объект
# с дополнительными возможностями (например, индексирование)
dir(nums)

['__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'count',
 'index',
 'start',
 'step',
 'stop']

### Лямбда-функции

In [43]:
# рассмотрим пример функции, которая преобразует
# количество секунд в формат времени (например, звучания трека)

def duration_string(seconds):
    return '{}:{:02d}'.format(seconds // 60, seconds % 60)

print(duration_string(411))
print(duration_string(120))

6:51
2:00


In [44]:
# небольшие функции можно объявлять "на ходу"
# это анонимные функции (лямбда-функции)

dur = lambda seconds: '{}:{:02d}'.format(seconds // 60, seconds % 60)

print(dur(411))
print(dur(120))

6:51
2:00


In [45]:
type(dur)

function

In [46]:
print(dur.__name__)
print(dur.__code__.co_varnames)

<lambda>
('seconds',)


Часто лямбда-функции используются в качестве параметров функций <b>map</b>, <b>filter</b>, <b>reduce</b>

In [47]:
# демонстрация filter()

files = ['1.wav', '2.mp3', '3.jpg', '4.png', '5.wav']

sound_files = filter(
    lambda f: f.endswith('wav') or f.endswith('mp3'), files)

print(sound_files)
print(list(sound_files))

['1.wav', '2.mp3', '5.wav']


In [48]:
# первым параметром функции filter() является любая функция;
# если функция используется одноразово и/или короткая телом,
# то лямбда хорошо подходит, иначе - подавать готовую функцию

def is_sound_file(filename):
    return filename.endswith('wav') or filename.endswith('mp3')

list(filter(is_sound_file, files))

['1.wav', '2.mp3', '5.wav']

In [49]:
# эквивалент filter() в виде генератора списка:
sound_files = [f for f in files 
               if f.endswith('wav') or f.endswith('mp3')]

sound_files

['1.wav', '2.mp3', '5.wav']

In [50]:
# конечно, можно написать и так:
sound_files = []
for f in files:
    if f.endswith('wav') or f.endswith('mp3'):
        sound_files.append(f)

sound_files

# НО НЕ НАДО, это ж не по-питоновски! ))

['1.wav', '2.mp3', '5.wav']

In [51]:
# демонстрация функции map()

# 1)
extensions = map(
    lambda f: f[f.index('.')+1:], files)

extensions = list(extensions)
print(extensions)

# 2)
print(list(map(str.upper, extensions)))

['wav', 'mp3', 'jpg', 'png', 'wav']
['WAV', 'MP3', 'JPG', 'PNG', 'WAV']


In [52]:
# эквивалент map() в виде генератора списка

# 1) 
extensions = [f[f.index('.')+1:] for f in files]
print(extensions)

# 2)
extensions = [ext.upper() for ext in extensions]
print(extensions)

['wav', 'mp3', 'jpg', 'png', 'wav']
['WAV', 'MP3', 'JPG', 'PNG', 'WAV']


In [53]:
# пример: отобразить числа в интервалы соответствующей длины

lengths = [3, 5, 1, 2]

intervals = map(lambda n: tuple(range(1, n+1)), lengths)
intervals = list(intervals)

print(intervals)
print()
for interval in intervals:
    print(interval)

[(1, 2, 3), (1, 2, 3, 4, 5), (1,), (1, 2)]

(1, 2, 3)
(1, 2, 3, 4, 5)
(1,)
(1, 2)


In [54]:
# Еще пример: посчитаем сумму цифр числа в декларативном стиле:
n = 123
sum(map(int, str(n)))

6

In [55]:
# NB. Функция reduce() в Python 2 является встроенной,
#     в Python 3 - кроется в модуле functools

from functools import reduce

s = range(1, 5)
reduce(lambda x, y: x + y, s)

10

In [56]:
# что здесь происходит?
words = ['He', 'evolves', 'fast', 'slowly']

sentence = reduce(
    lambda x, y: x + ' ' + y if x[-1] == y[0] else x, words)
sentence

'He evolves slowly'

### Еще элементы декларативного программирования в Python

In [57]:
# демонстрация функции zip()

name = ['John', 'Pete', 'Sue', 'Bob', 'Alice']
sex = ['m', 'm', 'f', 'm', 'f']
age = [27, 21, 25, 24, 23]

students = zip(name, sex, age)
print(students)
students = list(students)
print(students)

[('John', 'm', 27), ('Pete', 'm', 21), ('Sue', 'f', 25), ('Bob', 'm', 24), ('Alice', 'f', 23)]


In [58]:
# удобные и полезные функции any() и all():
help(any)
help(all)

Help on built-in function any in module builtins:

any(iterable, /)
    Return True if bool(x) is True for any x in the iterable.
    
    If the iterable is empty, return False.

Help on built-in function all in module builtins:

all(iterable, /)
    Return True if bool(x) is True for all values x in the iterable.
    
    If the iterable is empty, return True.



In [59]:
any(s[2] > 25 for s in students)

True

In [60]:
all(s[2] < 25 for s in students)

False

In [61]:
all(s[2] < 30 for s in students)

True

In [62]:
print(max(s[2] for s in students))
print(min(s[0] for s in students))

27
Alice


Продолжим рассмотрение возможностей модуля <b>itertools</b>, начатое в предыдущей лекции

In [63]:
from itertools import islice

list(islice(students, 1, 3))

[('Pete', 'm', 21), ('Sue', 'f', 25)]

In [64]:
from itertools import takewhile

some = takewhile(lambda s: len(s[0]) > 3, students)
list(some)

[('John', 'm', 27), ('Pete', 'm', 21)]

In [65]:
from itertools import dropwhile

some = dropwhile(lambda s: len(s[0]) > 3, students)
list(some)

[('Sue', 'f', 25), ('Bob', 'm', 24), ('Alice', 'f', 23)]

In [66]:
from itertools import groupby

students = sorted(students, key=lambda s: s[1])

for sex, info in groupby(students, lambda s: s[1]):
    print(list(info))

[('Sue', 'f', 25), ('Alice', 'f', 23)]
[('John', 'm', 27), ('Pete', 'm', 21), ('Bob', 'm', 24)]


В данном файле приводится сравнение кодов решения задач с помощью Python, C# LINQ и Java 8 Streams:

[Декларативная парадигма в Python, C# и Java (слайды)](Lec04 - Declarative Python, CSharp, Java.pdf)

Содержимое слайдов подробно разбирается на лекции.

### Декораторы

In [67]:
def copyright(func):
    
    def _wrapper():
        func()
        print('(c) Wise man')

    # декоратор возвращает функцию (callable)
    return _wrapper


# декорируем функцию
@copyright
def print_slogan():
    print('The truth will set you free')

    
print_slogan()

# происходит такой вызов:
# copyright(print_slogan())()

The truth will set you free
(c) Wise man


In [68]:
# пример декоратора функции, имеющей параметры:

def stringify(func):
    
    def _wrapper(*args, **kwargs):
        # преобразуем каждый аргумент функции к строке
        str_args = [str(arg) for arg in args]
        # вызовем исходную функцию со строковыми аргументами
        return func(*str_args, **kwargs)
        
    return _wrapper


# декорируем функцию
@stringify
def concat(x, y):
    return x + y


concat('Result: ', 10)

# происходит такой вызов:
# stringify(concat('OK', 10))()

# если убрать декоратор @stringify, при вызове будет ошибка

'Result: 10'

In [69]:
# Однакож:
print(concat.__name__)
print(concat.__qualname__)

# хотелось бы увидеть тут 'concat'...

_wrapper
stringify.<locals>._wrapper


In [70]:
# можно сделать что-то вроде этого:

def decorate(func):
    def _wrapped(*args, **kwargs):
        return func(args, kwargs)
    
    # явно скопировать необходимые переменные
    _wrapped.__name__ = func.__name__
    
    return _wrapped


@decorate
def baz():
    return 'baz'


print(baz.__name__)

baz


In [71]:
# но в Python есть для этого functools.wraps:

import functools

def stringify(func):
    
    @functools.wraps(func)
    def _wrapper(*args, **kwargs):
        # преобразуем каждый аргумент функции к строке
        str_args = [str(arg) for arg in args]
        # вызовем исходную функцию со строковыми аргументами
        return func(*str_args, **kwargs)
        
    return _wrapper


# декорируем функцию
@stringify
def concat(x, y):
    return x + y


print(concat('Result: ', 10))

print(concat.__name__)
print(concat.__qualname__)

Result: 10
concat
concat


In [72]:
# параметризированный декоратор
# (простейший способ его сделать - написать замыкание)

def copyright(year):
    
    def _decorator(func):
        
        def _wrapper(*args, **kwargs):
            func(*args, **kwargs)
            print('(c) {} - Wise man'.format(year))
            
        return _wrapper
    
    return _decorator


@copyright(2016)
def print_slogan():
    print('The truth will set you free')
    
    
print_slogan()    

The truth will set you free
(c) 2016 - Wise man


PS. Реализовать декоратор можно также с помощью класса с magic-методом ```__call__()```.

Здесь очень хорошо и подробно расписаны тонкости декораторов:

https://github.com/GrahamDumpleton/wrapt/tree/develop/blog

### Модуль functools

In [73]:
# модуль мал, да удал (wraps и reduce уже рассмотрены)
import functools

dir(functools)

['MappingProxyType',
 'RLock',
 'WRAPPER_ASSIGNMENTS',
 'WRAPPER_UPDATES',
 'WeakKeyDictionary',
 '_CacheInfo',
 '_HashedSeq',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_c3_merge',
 '_c3_mro',
 '_compose_mro',
 '_convert',
 '_find_impl',
 '_ge_from_gt',
 '_ge_from_le',
 '_ge_from_lt',
 '_gt_from_ge',
 '_gt_from_le',
 '_gt_from_lt',
 '_le_from_ge',
 '_le_from_gt',
 '_le_from_lt',
 '_lru_cache_wrapper',
 '_lt_from_ge',
 '_lt_from_gt',
 '_lt_from_le',
 '_make_key',
 'cmp_to_key',
 'get_cache_token',
 'lru_cache',
 'namedtuple',
 'partial',
 'partialmethod',
 'reduce',
 'singledispatch',
 'total_ordering',
 'update_wrapper',
 'wraps']

In [74]:
def prepare_email(to, sender, body):
    return 'To: {}\nFrom: {}\n\n{}'.format(to, sender, body)

print(prepare_email('joe@joe.com', 'me@at.me', 'Wazzup?'))

To: joe@joe.com
From: me@at.me

Wazzup?


In [75]:
# 1) partial - 
# создание "варианта" функции с частью закрепленных аргументов

from functools import partial

standard_email = partial(prepare_email, sender='me@at.me', body='Hello!')

print(standard_email('joe@joe.com'))

To: joe@joe.com
From: me@at.me

Hello!


In [76]:
boss_email = partial(prepare_email, 'boss@boss.bo', 'me@at.me')

print(boss_email('Let\'s talk about my salary!'))

To: boss@boss.bo
From: me@at.me

Let's talk about my salary!


In [77]:
print(standard_email.func)
print(standard_email.args)
print(standard_email.keywords)

<function prepare_email at 0x02D7FDB0>
()
{'sender': 'me@at.me', 'body': 'Hello!'}


In [78]:
# 2) singledispatch - 
# элемент обобщенного программирования в Python

from functools import singledispatch

@singledispatch
def product(a, b):
    return a * b
    
@product.register(list)
@product.register(tuple)
def _(a, b):
    return [i * b for i in a]


print(product([1, 2, 3], 10))
print(product(3.5, 2))
print(product(('a', 'b'), 3))

[10, 20, 30]
7.0
['aaa', 'bbb']


In [79]:
# 3) именованный кортеж

from functools import namedtuple

Circle = namedtuple('Circle', ['x', 'y', 'radius'])
Circle.__doc__

'Circle(x, y, radius)'

In [80]:
c = Circle(15, 15, 100)
c[1]

15

In [81]:
*center, radius = c
radius

100

In [82]:
# 4) мемоизация: LRU cache - Least Recently Used кеш

from functools import lru_cache

@lru_cache(maxsize=None)
def fib(n):
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)

[fib(n) for n in range(32)]

[0,
 1,
 1,
 2,
 3,
 5,
 8,
 13,
 21,
 34,
 55,
 89,
 144,
 233,
 377,
 610,
 987,
 1597,
 2584,
 4181,
 6765,
 10946,
 17711,
 28657,
 46368,
 75025,
 121393,
 196418,
 317811,
 514229,
 832040,
 1346269]

In [83]:
fib.cache_info()

CacheInfo(hits=60, misses=32, maxsize=None, currsize=32)

### Некоторые особенности замыканий

In [84]:
# Функция возвращает 5 функций, каждая из которых
# в разной степени расширяет строку ее копиями:

def string_extenders():
    return [lambda x: x * i for i in range(1, 6)]

# переменная i берется из скоупа замыкания
# и к моменту вызова каждой функции она равна 5
for extender in string_extenders():
    print(extender('Ha'))

HaHaHaHaHa
HaHaHaHaHa
HaHaHaHaHa
HaHaHaHaHa
HaHaHaHaHa


In [85]:
# чтобы обойти этот эффект, делаем копию i каждый раз
def string_extenders():
    return [lambda x, n=i: x * n for i in range(1, 6)]

for extender in string_extenders():
    print(extender('Ha'))

Ha
HaHa
HaHaHa
HaHaHaHa
HaHaHaHaHa


In [86]:
# или такой способ (в сугубо функциональном ключе):
from functools import partial
from operator import mul

def string_extenders():
    return [partial(mul, i) for i in range(1, 6)]

for extender in string_extenders():
    print(extender('Ha'))

Ha
HaHa
HaHaHa
HaHaHaHa
HaHaHaHaHa


<hr/>
### Подытожим:
База функционального программирования:
* чистые функции (pure functions) - детерменированные функции без побочных эффектов
* lambda-функции
* функции - это first-class объекты
* неизменяемость объектов и потоков данных
* рекурсии
* ленивые вычисления (в т.ч. генераторы)
* map, filter, reduce